# 夺旗赛 Capture The Flag

## 初始化 (使用上排目录栏的的 ▶️ 运行，▪️停止， ⟳ 重启）

In [8]:
import importlib
import lib.game_engine

# Force the reload manually
importlib.reload(lib.game_engine)

# Re-import the specific classes/functions
from lib.game_engine import GameMap, run_game_server

# Now initialize your objects
world = GameMap()

In [9]:
from IPython.display import clear_output
import os
import json
import random

## 以下是你要编写的代码。 
- start_game：初始化游戏。
- game_over：游戏结束。
- plan_next_actions：每一时刻，告诉你目前小人们的位置。
  
每次代码更新，一定要使用上排目录栏的的 ▶️ 运行

In [10]:
## 这是你要编写的策略
def start_game(req):
    """Called when the game begins."""
    world.init(req)
    print(f"Map initialized: {world.width}x{world.height}")

def game_over(req):
    """Called when the game ends."""
    print("Game Over!")

In [11]:
## 这是你要编写的策略。以下always_move_right和walk_to_first_flag_and_return是两个例子
def plan_next_actions(req):
    """
    Called every tick. 
    Return a dictionary: {"playerName": "direction"}
    direction is "up", "down", "right", "left, "" . "" means the player should stand still.
    """
    world.update(req)    
    world.show() 
    actions = dict()
    # TODO:请在这里写下你的代码来控制小人
    
    return actions


In [5]:
## 这是一个超简单策略：即把所有的小人都向右移动
## Python小测试 - 将以下程序改成：如果是L team，就往右走；如果是R team，就往左走。
def always_move_right(req):
    """
    Called every tick. 
    Return a dictionary: {"playerName": "direction"}
    direction is "up", "down", "right", "left, "" . "" means the player should stand still.
    """
    if not world.update(req):
        return

    world.show() 

    # List all players that can move freely (set `hasFlag=True`)
    my_players = world.list_players(mine=True, inPrison=False, hasFlag=None)

    actions = dict()
    for p in my_players:
        actions[p["name"]] = "right"

    return actions

In [12]:
## 这是一个超简单策略：所有小人都向着第一个小旗子走
## Python小测试 - 将以下程序改成：如果在对方的领地里，将对方的player位置设为extra_obstacles

def walk_to_first_flag_and_return(req):
    """
    Called every tick. 
    Return a dictionary: {"playerName": "direction"}
    direction is "up", "down", "right", "left, "" . "" means the player should stand still.
    """
    if not world.update(req):
        return

    # world.show() always show targets and prisons, regardless whether flags and players are not there or not
    world.show(flag_over_target=True, player_over_prison=True) 

    # List all players that can move freely (set `hasFlag=True`)
    my_players_go = world.list_players(mine=True, inPrison=False, hasFlag=False)
    my_players_return = world.list_players(mine=True, inPrison=False, hasFlag=True)
    # List a
    opponents = world.list_players(mine=False, inPrison=False, hasFlag=None)
    enemy_flags = world.list_flags(mine=False, canPickup=None)
    
    actions = {}
    
    # Everyone wo/ a flag rushes the first flag
    if enemy_flags:
        target_flag = enemy_flags[0]
        dest = (target_flag["posX"], target_flag["posY"])
        for p in my_players_go:
            start = (p["posX"], p["posY"])
            path = world.route_to(start, dest, extra_obstacles=[])
            
            if len(path) > 1:
                # Convert the next coordinate in path to a direction string
                next_step = path[1]
                actions[p["name"]] = GameMap.get_direction(start, next_step)
    # Everyone w/ a flag returns the target zone
    target_zone = list(world.list_targets(mine=True))[0]
    for p in my_players_return:
        start = (p["posX"], p["posY"])
        path = world.route_to(start, target_zone, extra_obstacles=[])
        if len(path) > 1:
            # Convert the next coordinate in path to a direction string
            next_step = path[1]
            actions[p["name"]] = GameMap.get_direction(start, next_step)
    return actions

## 运行以下cell启动你的server (使用上排目录栏的的 ▶️ 运行，使用▪️停止）
[*] 表示cell正在运行中

In [ ]:
# Change the port to match your game settings
import os

PORT_ID = 2 # 或者 2
PORT = "CTF_PORT_BACKEND" + str(PORT_ID)

try:
    PORT_VALUE = os.environ[PORT]
    print(f"PORT: {PORT_VALUE}")
except KeyError:
    print("Error: PORT environment variable not set.")
try:
    # 将`plan_fn=`改成你的plan_next_actions，如always_move_right, walk_to_first_flag_and_return 等等
    await run_game_server(PORT_VALUE, start_fn=start_game, plan_fn=walk_to_first_flag_and_return, end_fn=game_over)
except Exception as e:
    print(f"Server stopped: {e}")

    0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19
 0 ██ ██ ██ ██ ██ ██ ██ ██ ██ ██ ██ ██ ██ ██ ██ ██ ██ ██ ██ ██ 
 1 ██  .  .  .  .  .  .  . ██  .  .  .  .  .  . ██  .  . RF ██ 
 2 ██  .  .  . ██  .  .  . ██  .  .  .  .  .  .  .  .  . RF ██ 
 3 ██ LF  .  .  .  .  .  . ██  .  .  .  .  .  .  .  .  . RF ██ 
 4 ██ LF  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  . RF ██ 
 5 ██ LF  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  . ██ 
 6 ██ LF  .  .  .  .  .  .  .  .  . ██  .  .  .  .  .  .  . ██ 
 7 ██  .  .  .  .  .  .  .  .  .  .  .  .  .  . ██  .  .  . ██ 
 8 ██  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  . ██ 
 9 ██ RF RF TT ██  .  .  .  .  .  .  .  .  .  .  . TT TT TT ██ 
10 ██ TT L1 TT ██  .  .  .  .  . R0  .  .  .  .  . TT TT TT ██ 
11 ██ TT TT TT  .  .  .  .  .  .  .  .  .  .  .  . TT TT TT ██ 
12 ██  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  . ██ 
13 ██  .  .  .  .  .  .  .  .  . ██  .  .  .  .  .  .  .  . ██ 
14 ██  .  .  .  .  .  . ██  .  . ██  .  .